In [1]:
import argparse
import logging
import os
import torch
from torch.utils.data import DataLoader, random_split
import json
import rdkit.Chem as Chem
import rdkit
from rxntorch.containers.reaction import Rxn
from rxntorch.containers.molecule import Mol
from rxntorch.containers.dataset import RxnGraphDataset as RxnGD
from rxntorch.utils import collate_fn
import warnings
warnings.filterwarnings("ignore")
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from collections import defaultdict

from sklearn.preprocessing import StandardScaler
from collections import defaultdict

import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import LabelEncoder
import re
import glob

from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem import Descriptors



In [2]:
def isfloat(value):
    try:
        float(value)
        return True
    except ValueError:
        return False
    
def clean(num):
    if num is None:
        return 0
    if num=='[]':
        return ''
    elif math.isnan(float(num)):
        return 0
    else:
        return float(num)



In [3]:
def get_atom_mapping_doyle(mol_dir):
    
    smiles_mapping=defaultdict(dict) 
    if 'dy' in mol_dir:
        mol_dir=mol_dir.replace('.sdf','.mol')
        print(mol_dir)
    for mol_fn in glob.glob(mol_dir):
        atoms , labels, atom_mapping =[], [], []
        
        m = Chem.MolFromMolFile(mol_fn)
        smiles=Chem.MolToSmiles(m)
        mol_lines=open(mol_fn,'r').readlines()
        counter=defaultdict(int)
        for i,line in enumerate(mol_lines[4:]):
            l=re.sub(' +', ' ', line.strip('\n')).split(' ')
            if len(l)>10:
                if len(l)==17:
                    atom=l[4]
                    if atom !='' and '*' not in atom and 'H' not in atom:     
                        atoms.append(atom)
                        counter[atom]+=1
                        labels.append(atom+str(counter[atom]))
                #if "atom_labels" in line: # for Doyle
                    #labels=[i.strip('*') for i in mol_lines[i+1+4].strip('\n').split(' ') if ((i!='') and ('H' not in i))]
                    
        if len(atoms)==len(labels):
            for i in range(len(atoms)):
                atom_mapping.append((atoms[i],labels[i]))            
        else:
            print('Atoms and lables don\'t match')
            print(len(atoms),len(labels))
            atom_mapping=[]

        if smiles not in smiles_mapping:
            smiles_mapping[mol_fn.split('/')[-1].split('.')[0]]=atom_mapping
        else:
            print("key exsists")
    return smiles_mapping

In [11]:
descriptor_list=[x[0] for x in Descriptors._descList]

In [13]:
print(descriptor_list)

['MaxEStateIndex', 'MinEStateIndex', 'MaxAbsEStateIndex', 'MinAbsEStateIndex', 'qed', 'MolWt', 'HeavyAtomMolWt', 'ExactMolWt', 'NumValenceElectrons', 'NumRadicalElectrons', 'MaxPartialCharge', 'MinPartialCharge', 'MaxAbsPartialCharge', 'MinAbsPartialCharge', 'FpDensityMorgan1', 'FpDensityMorgan2', 'FpDensityMorgan3', 'BalabanJ', 'BertzCT', 'Chi0', 'Chi0n', 'Chi0v', 'Chi1', 'Chi1n', 'Chi1v', 'Chi2n', 'Chi2v', 'Chi3n', 'Chi3v', 'Chi4n', 'Chi4v', 'HallKierAlpha', 'Ipc', 'Kappa1', 'Kappa2', 'Kappa3', 'LabuteASA', 'PEOE_VSA1', 'PEOE_VSA10', 'PEOE_VSA11', 'PEOE_VSA12', 'PEOE_VSA13', 'PEOE_VSA14', 'PEOE_VSA2', 'PEOE_VSA3', 'PEOE_VSA4', 'PEOE_VSA5', 'PEOE_VSA6', 'PEOE_VSA7', 'PEOE_VSA8', 'PEOE_VSA9', 'SMR_VSA1', 'SMR_VSA10', 'SMR_VSA2', 'SMR_VSA3', 'SMR_VSA4', 'SMR_VSA5', 'SMR_VSA6', 'SMR_VSA7', 'SMR_VSA8', 'SMR_VSA9', 'SlogP_VSA1', 'SlogP_VSA10', 'SlogP_VSA11', 'SlogP_VSA12', 'SlogP_VSA2', 'SlogP_VSA3', 'SlogP_VSA4', 'SlogP_VSA5', 'SlogP_VSA6', 'SlogP_VSA7', 'SlogP_VSA8', 'SlogP_VSA9', 'TPSA'

In [3]:
def build_rdkit_features(smile,comp_name):
    descriptor_list=[x[0] for x in Descriptors._descList]
    calculator = MoleculeDescriptors.MolecularDescriptorCalculator(descriptor_list)
    rdkit_feature_names=[comp_name+'_'+j for j in calculator.GetDescriptorNames()]
    
    
    mol=Chem.MolFromSmiles(smile)
    mol= Chem.AddHs(mol)
    rdkit_feature_values= calculator.CalcDescriptors(mol)
    rdkit_feats = dict(zip(rdkit_feature_names,rdkit_feature_values))       
    

    return rdkit_feats

In [4]:
rows=set();y_factors=set()
data_type='az'
use_rdkit_features= False
if use_rdkit_features:
    print("Using rdkit features")
else:
    print("Not using rdkit features")
    
path = 'data/'+data_type+'/'
file_name = data_type+'_reactions_data.json'
mol_path=data_type+'_reaction_mols'

data_dict=defaultdict(lambda: defaultdict(float))
rxns = []; max_nbonds = 10   
                
with open(os.path.join(path, file_name)) as datafile:
    data = json.load(datafile)
    lines= data
    for line in lines:
        solvent_key='solvent' if 'solvent' in line.keys() else 'Solvent'
        base_key= 'Base' if 'Base' in line.keys() else 'base'
        

        
        if data_type=='az':
            name= reaction_num = line["reaction_Num"]
            r_yield=line['yield']['yield']
        
        if data_type in ['dy','su']:
            name= reaction_num = line["Id"]
            r_yield=line['yield']

        reactants=line['reactants']   
        product_smiles = line.get("product",{}).get('smiles','')
        base_smile = line.get(base_key,{}).get('smiles','')
        solvent_smile = 'CS(=O)C' if data_type=='dy' else line.get(solvent_key,[''])[0]
        if solvent_smile=='' or  solvent_smile==0:
            print(name)
            continue
        
        ###########other general reaction features 
        if data_type=='az': data_dict[name]['temperature'] = clean(line.get('temperature',0))
        elif data_type=='dy': data_dict[name]['temperature']=60.0
        elif data_type=='su': data_dict[name]['temperature']=100.0
            
        data_dict[name]['base_pka'] = clean(line.get(base_key,{}).get('Pka of Base-H',0))
        data_dict[name]['base_atom_cat'] = clean(line.get(base_key,{}).get('Atomic_number_Cation',0))   
        
        ##########other features in AZ data
        data_dict[name]['reaction_scale'] = clean(line.get('scale',0))
        data_dict[name]['base_amount'] = clean(line.get('base_amount',0))
        data_dict[name]['catalyst_amount'] = clean(line.get('catalyst_amount',0))
        data_dict[name]['reaction_volume'] = clean(line.get('volume',0))
        ##########
        #get solvent values
        solvent_vec= line.get(solvent_key,[])
        
        if solvent_vec !=[]:
            for i in range(1,len(solvent_vec)):
                data_dict[name]['solvent_'+str(i)]=float(solvent_vec[i])
        
        rxn = Rxn(name,reactants,solvent_smile,base_smile,r_yield)
        mol_reactants=rxn.mol_reactants

                
        for mol_idx in range(len(mol_reactants)):
            current_molecule = mol_reactants[mol_idx]
            category = current_molecule.category
            mol_smiles= current_molecule.smile
            
            if category=='Boronic Acid': category='Amine' #doing this helps with mapping reactions form Su to Dy and Az together
            
            #calculate rdkit features for reactants and integrate into data_dict
            if use_rdkit_features:
                rdkit_feats_curr_mol = build_rdkit_features(mol_smiles,category)
                for key, value in rdkit_feats_curr_mol.items():
                    data_dict[name][key] = value
            
            
            vib_modes = current_molecule.vib_modes

            atoms =current_molecule.atoms
            all_attributes=current_molecule.get_attributes().squeeze().tolist()
            weight,volume,surface_area,ovality,hardness,dipole_moment,electronegativity,HOMO,LUMO=all_attributes
            data_dict[name][category] = current_molecule.name
            data_dict[name][category +'_molecular_weight'] = round(float(weight),5)
            data_dict[name][category +'_molecular_volume'] = round(float(volume),5)
            data_dict[name][category +'_surface_area'] = round(float(surface_area),5)
            data_dict[name][category +'_ovality'] = round(float(ovality),5)
            data_dict[name][category +'_hardness'] = round(float(hardness),5)
            data_dict[name][category +'_dipole_moment'] = round(float(dipole_moment),5)
            data_dict[name][category +'_electronegativity'] = round(float(electronegativity),5)
            data_dict[name][category +'_E_HOMO'] = round(float(HOMO),5)
            data_dict[name][category +'_E_LOMO'] = round(float(LUMO),5)
            for n in range(len(vib_modes)):
                data_dict[name][category + '_V'+str(n)+'_frequency'] = round(float(vib_modes[n][0]),5)
                data_dict[name][category + '_V'+str(n)+'_intensity'] = round(float(vib_modes[n][1]),5)
            
            for atom in atoms:
                if 'H' not in atom['name']: #exculding hydrogen for now
                    if 'partial_charge' in atom:
                        data_dict[name][category+'_.'+atom['name']+'_electrostatic_charge']=round(float(clean(atom['partial_charge'])),5)
                                     
                    if 'nmr_shift' in atom:
                        if data_type=='az':
                            data_dict[name][category+'_.'+atom['name']+'_NMR_shift']= round(float(clean(atom['nmr_shift'])),5)
                        else:
                            data_dict[name][category+'_.'+atom['name']+'_NMR_shift']= round(float(clean(atom['nmr_shift'])),5)
                                                
            ######################### 
            #get rdkit fetures for base, solvent, product
            if use_rdkit_features:
                all_comps = [solvent_smile,base_smile,product_smiles]
                comp_map= {0:'solvent',1:'base',2:'product'}
                rdkit_feats_combined={}
                for i,smile in enumerate(all_comps):
                    if smile !='' or smile!=0:
                        rdkit_feats_combined = build_rdkit_features(smile,comp_map[i])            

                        for key, value in rdkit_feats_combined.items():
                            data_dict[name][key] = value
                
            ###############################
            
        if isfloat(r_yield):                
            data_dict[name]['yield'] = round(float(r_yield),5)      
        else:
            del data_dict[name]
            rows.add(name)
print("reactions with problematic yield: ",len(rows))
print("size of data_dict ",len(data_dict))

Not using rdkit features
reactions with problematic yield:  0
size of data_dict  750


In [6]:
df_o=pd.DataFrame.from_dict(data_dict, orient='index')
df_o=df_o.reset_index()
print("df_o shape:" ,df_o.shape)



df_o shape: (750, 383)


In [7]:
df_o=pd.DataFrame.from_dict(data_dict, orient='index')
df_o=df_o.reset_index()
print("df_o shape:" ,df_o.shape)


categorical =list( df_o.columns[ ~( (df_o.dtypes.values == np.dtype('float64')) | (df_o.dtypes.values == np.dtype('int64')))])
print("number of catgorical columns: ",len(categorical))
to_drop=list(df_o.columns[(df_o == 0).all()])+categorical#+['additive','base','aryl_halide','ligand']
scaler = StandardScaler()


df=df_o
df.drop(to_drop, axis=1, inplace=True)
curr_cols=list(df.columns)
if 'yield' in curr_cols:
    curr_cols.remove('yield')
new_cols=['yield']+curr_cols
print("number of new columns: ",len(new_cols))
len(curr_cols),len(curr_cols)
len(curr_cols)==len(curr_cols)

df_o shape: (750, 383)
number of catgorical columns:  4
number of new columns:  362


True

In [8]:
categorical

['catalyst_amount', 'amine', 'halide', 'ligand']

In [9]:
df.fillna(0, inplace=True)
df=df[new_cols]
#if data_type=='az' or data_type=='doyle':
df.iloc[:,1:] = scaler.fit_transform(df.iloc[:,1:].to_numpy())

print("Number of features after: ",len(list(df.columns)))

Number of features after:  362


### Generate a union of features from all datasets

In [38]:
features=list(df.columns)
feats=open('data/'+data_type+'/'+data_type+'_features.txt','w')
feats.write(','.join(map(str,features)))

14072

In [14]:
'data/'+data_type+'/'+data_type+'_features.txt'

'data/dy/dy_features.txt'

In [10]:
data_dict=df.transpose().to_dict(orient='list')
ext = '_data_raw_nordkit' if not use_rdkit_features else '_data_raw'

df.to_csv('data/'+data_type+'/'+data_type +ext +'.csv')
data_dict=df.transpose().to_dict(orient='list')
with open('data/'+data_type+'/'+data_type+'_reactions'+ext +'.txt','w') as f:
    for key,val in data_dict.items():
        f.write(str(key)+'\t'+','.join(map(str,val[1:]))+'\n')

### After running feature selection in 03_train_rf.ipynb, run the following to 
### select the subset of features identified by the RF model


In [45]:
df

,yield,solvent_Chi3n,Ligand_.P2_NMR_shift,Halide_LabuteASA,Halide_PEOE_VSA4,Amine_MaxEStateIndex,Halide_SMR_VSA10,base_Chi2n,base_VSA_EState9,solvent_SMR_VSA2,...,base_PEOE_VSA8,Halide_surface_area,Halide_Kappa2,solvent_MinEStateIndex,Halide_.I1_electrostatic_charge,Amine_SlogP_VSA1,base_SlogP_VSA1,Amine_.C4_NMR_shift,Amine_PEOE_VSA8,Amine_VSA_EState8
0,4.76,-0.806068,0.589613,-0.818947,-0.505291,-0.638975,-0.707489,-1.019992,0.263761,1.648479,...,-0.606478,-0.715967,-0.839860,1.199103,0.505291,-0.548382,-0.488376,0.764105,-0.457358,-0.809802
1,4.12,-0.806068,0.589613,-0.818947,-0.505291,-0.638975,-0.707489,-1.019992,0.263761,1.648479,...,-0.606478,-0.715967,-0.839860,1.199103,0.505291,-0.548382,-0.488376,0.764105,-0.457358,-0.809802
2,2.58,-0.806068,0.589613,-0.818947,-0.505291,-0.638975,-0.707489,-1.019992,0.263761,1.648479,...,-0.606478,-0.715967,-0.839860,1.199103,0.505291,-0.548382,-0.488376,0.764105,-0.457358,-0.809802
3,4.44,-0.806068,0.589613,-0.818947,-0.505291,-0.638975,-0.707489,-1.019992,0.263761,1.648479,...,-0.606478,-0.715967,-0.839860,1.199103,0.505291,-0.548382,-0.488376,0.764105,-0.457358,-0.809802
4,1.95,-0.806068,0.589613,-0.818947,-0.505291,-0.638975,-0.707489,-1.019992,0.263761,1.648479,...,-0.606478,-0.715967,-0.839860,1.199103,0.505291,-0.548382,-0.488376,0.764105,-0.457358,-0.809802
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4538,69.71,0.044852,0.589613,2.827300,-0.505291,-0.670728,-0.487853,0.538201,1.185961,-0.787894,...,-0.606478,2.140026,1.524634,-1.132745,0.505291,-0.780314,2.230782,-1.262244,0.076193,-0.821602
4539,47.21,0.044852,-2.301713,2.827300,-0.505291,-0.670728,-0.487853,0.538201,1.185961,-0.787894,...,-0.606478,2.140026,1.524634,-1.132745,0.505291,-0.780314,2.230782,-1.262244,0.076193,-0.821602
4540,0.00,0.044852,0.589613,2.827300,-0.505291,-0.670728,-0.487853,0.538201,1.185961,-0.787894,...,-0.606478,2.140026,1.524634,-1.132745,0.505291,-0.780314,2.230782,-1.262244,0.076193,-0.821602
4541,31.44,0.044852,-1.160679,2.827300,-0.505291,-0.670728,-0.487853,0.538201,1.185961,-0.787894,...,-0.606478,2.140026,1.524634,-1.132745,0.505291,-0.780314,2.230782,-1.262244,0.076193,-0.821602


In [43]:
selected_feats=open('data/'+data_type+'/'+data_type+'_selected_features.txt','r').readline().strip().split(',')
selected_feats = ['yield']+ selected_feats 
df=df[selected_feats]

In [44]:
data_dict=df.transpose().to_dict(orient='list')
df.to_csv('data/'+data_type+'/'+data_type+'_data.csv')
data_dict=df.transpose().to_dict(orient='list')
with open('data/'+data_type+'/'+data_type+'_reactions_data.txt','w') as f:
    for key,val in data_dict.items():
        f.write(str(key)+'\t'+','.join(map(str,val[1:]))+'\n')

### Get the ligands in Suzuki 

In [ ]:
data_type='dy'
with open(os.path.join('data/', data_type+'_reactions_data.json')) as datafile:
    data = json.load(datafile)
    lines= data
atoms1=[]   
atoms2=[]
line=lines
for line in lines[0:3]:
    reactants=line['reactants']
    
    rxn = Rxn(line['Id'],reactants,line['yield'])
    smiles= rxn.reactants_smile
    print(smiles)
    mol=Chem.MolFromSmiles(smiles)
    for atom in mol.GetAtoms():
        atoms1.append(atom.GetSymbol())
    for react in smiles.split('.'):
        mol=Chem.MolFromSmiles(react)
        smile=Chem.MolToSmiles(mol)
        print(smile)
        for atom in mol.GetAtoms():
            atoms2.append(atom.GetSymbol())

In [ ]:
data_type='su'
with open(os.path.join('data/', data_type+'/'+data_type+'_reactions_data.json')) as datafile:
    data = json.load(datafile)
    lines= data
atoms1=[]   
atoms2=[]
ligands=set()
line=lines
for line in lines:
    reactants=line['reactants']
    for reactant in reactants:
        key='Category' if 'Category' in reactant else 'category'
        if reactant[key]=='Ligand':
            ligands.add(reactant['smiles'])

{'C1=C[C@H]2[Fe][C@@H]1[C]2P(c1ccccc1)c1ccccc1.CC(C)(C)P(C1=CCC=C1)C(C)(C)C',
 'C1CCC(P(C2CCCCC2)C2CCCCC2)CC1',
 'CC(C)(C)P(C(C)(C)C)C(C)(C)C',
 'CC(C)(C)P([C]1C=C[CH][C@@H]1[Fe]C1C=CC=C1P(C(C)(C)C)C(C)(C)C)C(C)(C)C',
 'CC(C)c1cc(C(C)C)c(-c2ccccc2P(C2CCCCC2)C2CCCCC2)c(C(C)C)c1',
 'CC1(C)c2cccc(P(c3ccccc3)c3ccccc3)c2Oc2c(P(c3ccccc3)c3ccccc3)cccc21',
 'CCCCP(C12CC3CC(CC(C3)C1)C2)C12CC3CC(CC(C3)C1)C2',
 'CN(C)Cc1ccc(P(C(C)(C)C)C(C)(C)C)cc1',
 'COc1cccc(OC)c1-c1ccccc1P(C1CCCCC1)C1CCCCC1',
 'Cc1ccccc1P(c1ccccc1C)c1ccccc1C',
 'c1ccc(P(c2ccccc2)c2ccccc2)cc1'}

